In [1]:
import requests, json, sys, folium
import pandas as pd
from postcodes_api import PostcodeApi

In [2]:
sch_stat = pd.read_excel('Datasets/scottish_schools_stats.xlsx')
sch_info = pd.read_excel('Datasets/scottish_schools_contact.xlsx', sheet_name='Open Schools')
dep = pd.read_excel('Datasets/postcode_deprivation.xlsx')

In [3]:
dep_rates = {}

for p,d in dep.values:
    dep_rates[p] = d

In [4]:
sch_stat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2461 entries, 0 to 2460
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Local Authority  2461 non-null   object
 1   SeedCode         2461 non-null   int64 
 2   School Name      2461 non-null   object
 3   School Type      2461 non-null   object
 4   Total pupils     2461 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 96.3+ KB


In [5]:
sch_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2458 entries, 0 to 2457
Data columns (total 19 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   Seed Code                                2458 non-null   int64 
 1   LA Name                                  2458 non-null   object
 2   Centre Type                              2458 non-null   object
 3   School Name                              2458 non-null   object
 4   Address Line1                            2458 non-null   object
 5   Address Line2                            2457 non-null   object
 6   Address Line3                            2458 non-null   object
 7   Post Code                                2458 non-null   object
 8   Unique Property Reference Number (UPRN)  2458 non-null   int64 
 9   Email                                    2458 non-null   object
 10  Phone Number                             2458 non-null   obj

In [6]:
sch_info = sch_info.rename(columns={'Seed Code' : 'SeedCode'})

In [7]:
sch_df = sch_info.merge(sch_stat, on='SeedCode', how='right')

In [8]:
sch_df.isna().sum()
sch_df.dropna(inplace=True)

In [9]:
sch_jsn = json.loads(sch_df.to_json(orient='records'))

In [10]:
postcodes = [sch['Post Code'] for sch in sch_jsn]

In [11]:
sch_loc_info = {'locs' : [], 'cities' : [], 'zones' : []}

for i in range(100,len(postcodes),100):

    sch_loc_info_100 = PostcodeApi().get_bulk_pos_info(postcodes[i-100:i])
    sch_loc_info['locs'] += sch_loc_info_100['locs']
    sch_loc_info['cities'] +=  sch_loc_info_100['cities']
    sch_loc_info['zones'] += sch_loc_info_100['zones']

    if (len(postcodes) - i) < 100:
        sch_loc_info_less = PostcodeApi().get_bulk_pos_info(postcodes[i:len(postcodes)])
        
        sch_loc_info['locs'] += sch_loc_info_less['locs']
        sch_loc_info['cities'] +=  sch_loc_info_less['cities']
        sch_loc_info['zones'] += sch_loc_info_less['zones']

In [12]:
sch_df = pd.concat([pd.DataFrame(sch_jsn), pd.DataFrame(sch_loc_info)], axis=1)

In [21]:
null_indices = sch_df[sch_df.isnull().any(axis=1)].index

In [26]:
for i in null_indices:

    ps = sch_df['Post Code'].iloc[i]

    PostcodeApi().get_pos_info(ps)

The postcode DD8 5BR is terminated.
The postcode DG3 5DS is terminated.
The postcode DG3 5DS is terminated.
The postcode DD5 3AE is terminated.
The postcode EH33 2LX is terminated.
Non-valid post code: EH7 4TN
The postcode KY8 1HL is terminated.
The postcode G20 8LY is terminated.
The postcode G33 3LT is terminated.
The postcode G33 3LT is terminated.
The postcode G21 1NL is terminated.
The postcode G33 4SA is terminated.
The postcode G2 4PF is terminated.
The postcode ML2 0LS is terminated.
The postcode ML2 0LS is terminated.
The postcode PH15 2DU is terminated.
The postcode PH15 2DU is terminated.
The postcode TD11 3QQ is terminated.
The postcode TD4 6HF is terminated.
The postcode KA7 1HX is terminated.
The postcode KA26 9AQ is terminated.
The postcode KA7 2PG is terminated.
The postcode ML9 1NJ is terminated.
The postcode G74 3QT is terminated.


In [32]:
sch_df.head()

,SeedCode,LA Name,Centre Type,School Name_x,Address Line1,Address Line2,Address Line3,Post Code,Unique Property Reference Number (UPRN),Email,...,Gaelic Unit,Integrated Special Unit,Denomination,Local Authority,School Name_y,School Type,Total pupils,locs,cities,zones
0,1002929,Aberdeen City,Local Authority,Brimmond School,Netherhills Place,Bucksburn,ABERDEEN,AB21 9DF,9.051145e+09,brimmondschool@aberdeencity.gov.uk,...,No,Yes,Non-denominational,Aberdeen City,Brimmond School,Primary,425,"(57.174405, -2.186752)",Aberdeen City,Aberdeen North
1,1004247,Aberdeen City,Local Authority,Countesswells School,Airyhall School,Countesswells Road,ABERDEEN,AB15 8AD,9.051019e+09,CountesswellsSchool@aberdeencity.gov.uk,...,No,No,Non-denominational,Aberdeen City,Countesswells School,Primary,110,"(57.132094, -2.149723)",Aberdeen City,Aberdeen South
2,5230420,Aberdeen City,Local Authority,Riverbank School,Dill Road,Tillydrone,ABERDEEN,AB24 2XL,9.051055e+09,riverbank@aberdeencity.gov.uk,...,No,Yes,Non-denominational,Aberdeen City,Riverbank School,Primary,332,"(57.172511, -2.119037)",Aberdeen City,Aberdeen North
3,5230624,Aberdeen City,Local Authority,Westpark School,Cruden Crescent,Northfield,ABERDEEN,AB16 7JD,9.051027e+09,westpark@aberdeencity.gov.uk,...,No,No,Non-denominational,Aberdeen City,Westpark School,Primary,242,"(57.163969, -2.168203)",Aberdeen City,Aberdeen North
4,5230829,Aberdeen City,Local Authority,Heathryburn School,Howes Road,0,ABERDEEN,AB16 7RW,9.051317e+09,heathryburn@aberdeencity.gov.uk,...,No,No,Non-denominational,Aberdeen City,Heathryburn School,Primary,306,"(57.165031, -2.173929)",Aberdeen City,Aberdeen North


In [35]:
import folium
from folium.plugins import HeatMap

# Create a map centered at a specific location
m = folium.Map(location=[55.941457, -3.205744], zoom_start=12)  # London, UK

# Create a custom color scale from light to dark blue
colors = {
    1: '#08306b',  # Light blue
    2: '#08519c',
    3: '#3182bd',
    4: '#63b7f4',
    5: '#a6e1fa'   # Dark blue (most deprived)
}

l = len(sch_df)

# Create circles and digits for each data point
for i in range(len(sch_df)):

    l -= 1
    print(f'{l} remains')

    school = sch_df['School Name_x'].iloc[i]
    pos = sch_df['Post Code'].iloc[i]
    loc = sch_df['locs'].iloc[i]
    city = sch_df['cities'].iloc[i]
    zone = sch_df['zones'].iloc[i]
    pupils = sch_df['Total pupils'].iloc[i]

    try:
        mag = dep_rates[pos] # magnitute
    except:
        mag = 3 # average
    
    folium.CircleMarker(
        location=loc,
        radius=25//mag,
        color=colors[mag],
        fill=True,
        fill_opacity=0.8,
    ).add_to(m)

    popup_html = f"""
    <h3>{school}</h3>
    <p><strong>Local Authority:</strong> {city}</p>
    <p><strong>Zone:</strong> {zone}</p>
    <p><strong>Pupils:</strong> {pupils}</p>
    <p><strong>Deprivation:</strong> {mag}</p>"""

    folium.Marker(
        location=loc,
        popup=folium.Popup(popup_html, max_width=150),
        icon=folium.DivIcon(html=f'<div style="width: 0px; height: 0px;"></div>'),
    ).add_to(m)

# Create a HeatMap layer using some sample data points
# heat_data = [[51.5074, -0.1278, 0.5], [51.5099, -0.1337, 0.2], [51.5113, -0.1219, 0.3]]
# HeatMap(heat_data).add_to(m)

# Create a custom HTML legend
legend_html = """
<div style="position: fixed; top: 10px; right: 10px; background-color: white; padding: 10px; border: 2px solid black; z-index: 1000;">
    <p><strong>Legend</strong></p>
    <p><span style="color: black;"><span style="background-color: #08306b; width: 20px; height: 20px; display: inline-block;"></span> 1 - Most Deprived</span></p>
    <p><span style="color: black;"><span style="background-color: #08519c; width: 20px; height: 20px; display: inline-block;"></span> 2</span></p>
    <p><span style="color: black;"><span style="background-color: #3182bd; width: 20px; height: 20px; display: inline-block;"></span> 3</span></p>
    <p><span style="color: black;"><span style="background-color: #63b7f4; width: 20px; height: 20px; display: inline-block;"></span> 4</span></p>
    <p><span style="color: black;"><span style="background-color: #a6e1fa; width: 20px; height: 20px; display: inline-block;"></span> 5 - Least Deprived</span></p>
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))


# Display the map (optional, this works in Jupyter Notebook)
m

2459 remains
2458 remains
2457 remains
2456 remains
2455 remains
2454 remains
2453 remains
2452 remains
2451 remains
2450 remains
2449 remains
2448 remains
2447 remains
2446 remains
2445 remains
2444 remains
2443 remains
2442 remains
2441 remains
2440 remains
2439 remains
2438 remains
2437 remains
2436 remains
2435 remains
2434 remains
2433 remains
2432 remains
2431 remains
2430 remains
2429 remains
2428 remains
2427 remains
2426 remains
2425 remains
2424 remains
2423 remains
2422 remains
2421 remains
2420 remains
2419 remains
2418 remains
2417 remains
2416 remains
2415 remains
2414 remains
2413 remains
2412 remains
2411 remains
2410 remains
2409 remains
2408 remains
2407 remains
2406 remains
2405 remains
2404 remains
2403 remains
2402 remains
2401 remains
2400 remains
2399 remains
2398 remains
2397 remains
2396 remains
2395 remains
2394 remains
2393 remains
2392 remains
2391 remains
2390 remains
2389 remains
2388 remains
2387 remains
2386 remains
2385 remains
2384 remains
2383 remains

ValueError: CircleMarker location must be assigned when added directly to map.

In [11]:
l = len(sch_jsn)

for sch in sch_jsn:
    l -= 1
    print(l)
    pos = sch['Post Code']
    PostcodeApi().get_pos_info(pos)

2459
2458
2457
2456


KeyboardInterrupt: 

### Send bulk request

In [9]:
import requests

# Define the URL
url = "https://api.postcodes.io/postcodes"

# Define the JSON payload
data = {
    "postcodes": ["PR3 0SG"]
}

# Set the headers
headers = {
    "Content-Type": "application/json"
}

# Send the POST request
response = requests.post(url, json=data, headers=headers)

# Check the response
if response.status_code == 200:
    # The request was successful, and you can parse the response JSON
    result = response.json()
    print(result)
else:
    # Handle the error
    print(f"Error: {response.status_code} - {response.text}")

{'status': 200, 'result': [{'query': 'PR3 0SG', 'result': {'postcode': 'PR3 0SG', 'quality': 1, 'eastings': 351012, 'northings': 440302, 'country': 'England', 'nhs_ha': 'North West', 'longitude': -2.746251, 'latitude': 53.856635, 'european_electoral_region': 'North West', 'primary_care_trust': 'North Lancashire Teaching', 'region': 'North West', 'lsoa': 'Wyre 006A', 'msoa': 'Wyre 006', 'incode': '0SG', 'outcode': 'PR3', 'parliamentary_constituency': 'Wyre and Preston North', 'admin_district': 'Wyre', 'parish': 'Myerscough and Bilsborrow', 'admin_county': 'Lancashire', 'date_of_introduction': '200705', 'admin_ward': 'Brock with Catterall', 'ced': 'Wyre Rural East', 'ccg': 'NHS Lancashire and South Cumbria', 'nuts': 'Lancaster and Wyre', 'pfa': 'Lancashire', 'codes': {'admin_district': 'E07000128', 'admin_county': 'E10000017', 'admin_ward': 'E05009934', 'parish': 'E04005340', 'parliamentary_constituency': 'E14001057', 'ccg': 'E38000226', 'ccg_id': '02M', 'ced': 'E58000832', 'nuts': 'TLD4

In [11]:
result['result']

[{'query': 'PR3 0SG',
  'result': {'postcode': 'PR3 0SG',
   'quality': 1,
   'eastings': 351012,
   'northings': 440302,
   'country': 'England',
   'nhs_ha': 'North West',
   'longitude': -2.746251,
   'latitude': 53.856635,
   'european_electoral_region': 'North West',
   'primary_care_trust': 'North Lancashire Teaching',
   'region': 'North West',
   'lsoa': 'Wyre 006A',
   'msoa': 'Wyre 006',
   'incode': '0SG',
   'outcode': 'PR3',
   'parliamentary_constituency': 'Wyre and Preston North',
   'admin_district': 'Wyre',
   'parish': 'Myerscough and Bilsborrow',
   'admin_county': 'Lancashire',
   'date_of_introduction': '200705',
   'admin_ward': 'Brock with Catterall',
   'ced': 'Wyre Rural East',
   'ccg': 'NHS Lancashire and South Cumbria',
   'nuts': 'Lancaster and Wyre',
   'pfa': 'Lancashire',
   'codes': {'admin_district': 'E07000128',
    'admin_county': 'E10000017',
    'admin_ward': 'E05009934',
    'parish': 'E04005340',
    'parliamentary_constituency': 'E14001057',
   